In [5]:
 require 'rest-client'
  def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
  
    response = RestClient::Request.execute({
      method: :get,
      url: url.to_s,
      user: user,
      password: pass,
      headers: headers})

  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response  
  rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
  rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
    
end 

:fetch

In [ ]:
class Interactions < Gene
  

In [ ]:
class Gene
  
  attr_accessor :gene_ID  
  attr_accessor :interactions
  #attr_accessor :go_ID
  #attr_accessor :kegg_pathways
  
  def initialize (params = {})
    
    @gene_ID = params.fetch(:gene_ID, "000000000")
    @interaction = params.fetch(:interactions, Array.new)
    #@go_ID = params.fetch(:go_ID, "00/00/0000")
    #@kegg_pathways = params.fetch(:kegg_pathways, 'unknown place')
  end
  
  def search_interactions(some_gene)
      link = fetch("http://bar.utoronto.ca:9090/psicquic/webservices/current/search/query/#{some_gene}");
      if link
        body = link.to_s.split("\n")
        body.each do |interaction|
          line = interaction.split("\t")
          agi1 = line[2].split(":")[1].upcase
          agi2 = line[3].split(":")[1].upcase
          tax1 = line[9].split(":")[1]
          tax2 = line[10].split(":")[1]
          score = line[14].split(":")[1]
          if tax1.to_i != 3702 or tax2.to_i !=3702 # Filtering by species (Arabidopsis thaliana).
            next
          elsif score.to_f < 0.45 # Filtering by an intermediate score (DOI: 10.1093/database/bau131).
            next
          elsif some_gene.to_s == agi1.to_s && some_gene.to_s == agi2.to_s # Discarding interactions beteen the same gene.         
            next
          else
            #interaction = OpenStruct.new
            case
            when agi1.to_s == some_gene.to_s
              ggi = [some_gene, agi2]
              puts some_gene
              puts agi1
              puts agi2
              puts ggi
              #puts agi1.to_s
              #puts interaction.interactions
              #search_interactions(agi2)
            else
              ggi = [some_gene, agi1]
              puts some_gene
              puts agi1
              puts agi2
              puts ggi
              #puts interaction.interactions
              #search_interactions(agi1) 
            end
            #interaction.interactions << ggi
          end
        end
      else
        puts "the Web call failed - see STDERR for details..."
    end
  end
end

In [ ]:
file = File.open("Arabidopsis_prueba.txt", "r").each do |gene_id|
  genes = Gene.new(:gene_ID => gene_id.strip.upcase)
  genes.search_interactions(genes.gene_ID)
end
file.close

In [ ]:
  def find_go(gene1,a)
    res = fetch("http://togows.org/entry/ebi-uniprot/#{gene1}/dr.json");
    if res
      data = JSON.parse(res.body)
      if data[0] and data[0]["GO"]
        data[0]["GO"].each do |go|
          if go[1].match(/P:/)
            a << go[0] + " " + go[1]
            puts a
          else 
            next
          end
        end
      end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end
b = []

In [ ]:
find_go("At2g09990",b)

In [1]:
  def find_go(gene1)
    res = fetch("http://togows.org/entry/ebi-uniprot/#{gene1}/dr.json");
    if res
      data = JSON.parse(res.body)
      final_go = []
      if data[0] and data[0]["GO"]
        data[0]["GO"].each do |go|
          if go[1].match(/P:/)
            final_go << go[0] + " " + go[1]
            puts final_go
          else 
            return
          end      
        end
      end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end

:find_go

In [2]:
  def find_kegg(gene2)
    ros = fetch("http://togows.org/entry/kegg-genes/ath:#{gene2}/pathways.json");
    if ros
      data = JSON.parse(ros.body)
      final_kegg = []
      if data[0]
        data[0].each do |kegg|
            final_kegg = kegg[0]+":"+kegg[1]
          puts final_kegg
          
        end
      end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end

:find_kegg

In [ ]:
find_kegg("AT4g17030")

In [ ]:
class ListNetworks
  
  @@all_interactions = Array.new
  attr_accessor :network_ID
  attr_accessor :interactors
  attr_accessor :network_go
  attr_accessor :network_kegg
  
  def initialize (params = {})
    @network_ID = params.fetch(:network_ID, "000000000")
    @interactors = params.fetch(:interactors, [])
    @network_go = params.fetch(:network_go, Array.new)
    @network_kegg = params.fetch(:network_kegg, Array.new)
       
    @@all_interactions << self # Variable containing all the objects of this class
    
  end  
  
    def self.find_go(gene1, to_which_class) # Function to find the "GO: Biological Process" of the genes
    res = fetch("http://togows.org/entry/ebi-uniprot/#{gene1}/dr.json"); # Link to search for interctions indicated in "http://togows.dbcls.jp/"
    if res
      data = JSON.parse(res.body)
      if data[0] and data[0]["GO"] # Avoiding errors due to non-existent web page or GO terms for the gene
        #final_go = []
      data[0]["GO"].each do |go|
        if go[1].match(/P:/) # Matching the "P", term for the Biological Process
          go_term = [go[0] + " " + go[1]]
          case
            when to_which_class == 1
              if not (@go_p.include?(go_term))
                @go_p << go_term
              end
              when to_which_class == 2
                ListNetworks.gop_to_ListNetworks(go_term)
          end
        else
          next
        end
      end
    end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end
  
  def self.find_kegg(gene2, to_which_class) # Function to find the "KEEG Pathways" of the genes
    ros = fetch("http://togows.org/entry/kegg-genes/ath:#{gene2}/pathways.json"); # Link to search for interctions indicated in "http://togows.dbcls.jp/"
    if ros
      data = JSON.parse(ros.body)
      if data[0] # Avoiding errors due to non-existent web page
        data[0].each do |kegg|
          kegg_term = [kegg[0]+":"+kegg[1]]
          case
            when to_which_class == 1
              if not (@kegg_pathways.include?(kegg_term))
                @kegg_pathways <<  kegg_term
              end
            when to_which_class == 2
              ListNetworks.kegg_to_ListNetworks(kegg_term)
            end
          end
        end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end

 def ListNetworks.gop_to_ListNetworks(go_process)
    if @network_go == nil or not (@network_go.include?(go_process))
      @network_go = [go_process]
      puts @network_go
    end
  end
        
    def ListNetworks.kegg_to_ListNetworks(kegg_id_path)
      if @network_kegg == nil or not (@network_kegg.include?(kegg_id_path))
        @network_kegg << [kegg_id_path]
        puts @network_kegg
      end
    end

def ListNetworks.is_in_the_list(gene_to_check) # Function to create networks for genes of the input list that interact between them
    if $list_of_genes.include?(gene_to_check) 
      if @network_ID == nil or (not @network_ID.include?("AT1G07010"))
        ListNetworks.new(:network_ID => "AT1G07010",
                         :interactors => ["AT1G07010"],
                         :network_go => nil,
                         :network_kegg => nil)
        puts @interactors
        find_go(gene_to_check, 2)
        find_kegg(gene_to_check, 2)
      else @network_ID == $AGI and not (@interactors.include?(gene_to_check))
        @interactors << gene_to_check
        puts @interactors
        find_go(gene_to_check, 2)
        find_kegg(gene_to_check, 2)
      end
    end
  end
end
  $list_of_genes = ["AT1G07010", "AT4G17030", "AT1g17380"]

In [ ]:
ListNetworks.is_in_the_list("AT4G17030")

In [4]:
require 'term/ansicolor'
include Term::ANSIColor
puts "hola".bold

hola


In [3]:
  def list_interactions(gene_to_check)
      find_go(gene_to_check)
      find_kegg(gene_to_check)
      return "is in the list"
    end

:list_interactions

In [6]:
list_interactions("AT4G17030")

"is in the list"